In [1]:
import awswrangler  as wr
import pandas as pd
import pymysql
import datetime
import json


In [2]:
sql_v1 = ''' 
select
mno
, mvno
, date_diff( 'hour', reg_ts , completed_at )/24 as diff_hour
, case when order_type= 'self' then  date_diff( 'hour', reg_ts , completed_at )/24 end as self_completed
,case when order_type = 'register' then date_diff( 'hour', reg_ts , completed_at )/24 end as regi_completed
from moyo_order.mobile_plan_order
where 1=1
    and date(reg_ts) between date('2023-05-01') and date('2023-06-01')
'''

In [3]:
df_v1= wr.athena.read_sql_query(sql=sql_v1, database="moyo_order")

In [4]:
df_v1.head()


,mno,mvno,diff_hour,self_completed,regi_completed
0,LGU,smartel,<NA>,<NA>,<NA>
1,LGU,사람과연결,<NA>,<NA>,<NA>
2,LGU,insmobile,0,0,<NA>
3,KT,pointplus,11,<NA>,11
4,LGU,이지모바일,4,<NA>,4


In [5]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27074 entries, 0 to 27073
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   mno             26950 non-null  string
 1   mvno            27074 non-null  string
 2   diff_hour       17386 non-null  Int64 
 3   self_completed  6244 non-null   Int64 
 4   regi_completed  11142 non-null  Int64 
dtypes: Int64(3), string(2)
memory usage: 1.1 MB


In [7]:
df_v1.dropna(inplace=False)

,mno,mvno,diff_hour,self_completed,regi_completed


In [13]:
df_v1.diff_hour.dropna(inplace=False)

2         0
3        11
4         4
6         7
7         8
         ..
27067     7
27069     1
27070     0
27071     8
27073    18
Name: diff_hour, Length: 17386, dtype: Int64

In [ ]:
# apply_plan 기준으로 분기 만들어서 max_diff_min 의 ecdf 그려줘 
# ecdf plot 
# Path: Workbook/user_behavier_churn _and_duplicated_auth/user_duplicated_auth.ipynb
import numpy as np
import matplotlib.pyplot as plt

# Assuming df is your pandas DataFrame, and 'dimension' is your categorical variable

for category in df_v1['apply_plan'].unique():
    # Select data for this category
    data = df_v1[df_v1['apply_plan'] == category]

    # Compute ECDF
    x = np.sort(data['max_diff_min'])
    y = np.arange(1, len(x) + 1) / len(x)

    # Plot ECDF
    plt.plot(x, y, label=category, alpha = 0.5)

plt.xlabel('Your Numeric Column')
plt.ylabel('ECDF')
# plt.axvline(x=60, color='r', linestyle='--')
plt.xlim(-10, 100)
plt.legend()
plt.show()

